# 1. Import libraries

In [29]:
import requests
from pandas import json_normalize
import numpy as np 
import pandas as pd
import os
import sys
sys.path.append("../")

# Importing the cleaning functions:
import src.cleaning_functions as cf

# 2. Import data

In [30]:
smart = pd.read_csv("../Data/clean_smart.csv",encoding = "utf-8")

In [31]:
#Now we use the string method replace to match dataframe names with APIs:
smart["Country"] = smart["Country"].str.replace("United States", "United States Of America")

# 3. Request data from APIS

## 3.1 API COUNTRY - CONTINENTS

In [32]:
import requests
import os
from dotenv import load_dotenv
load_dotenv()
token = os.getenv("token")

In [33]:
import requests

url = "https://referential.p.rapidapi.com/v1/country"

querystring = {"fields":"currency,currency_num_code,currency_code,continent_code,currency,iso_a3,dial_code"}

headers = {
    'x-rapidapi-host': "referential.p.rapidapi.com",
    'x-rapidapi-key': token
    }

response = requests.request("GET", url, headers=headers, params=querystring)


In [34]:
#response.json()

In [35]:
def continent(country):
    for dicc in response.json():
        if country == "China":
            return "AS"
        elif country == dicc["value"]:
            return dicc["continent_code"]

In [36]:
smart["Continent"] = smart.Country.apply(continent)

In [37]:
smart["Continent"] = smart["Continent"].str.replace("NA", "North America")
smart["Continent"] = smart["Continent"].str.replace("OC", "Oceania")
smart["Continent"] = smart["Continent"].str.replace("EU", "Europe")
smart["Continent"] = smart["Continent"].str.replace("AS", "Asia")

## 3.1 API LIFE QUALITY

In [38]:
cities = smart["City"] = smart["City"].apply(cf.lowercase)

In [39]:
scores =[]
for city in cities:
    try: 
        response = requests.get(f"https://api.teleport.org/api/urban_areas/slug:{city}/scores/").json()
        scores.append(response["teleport_city_score"])
    except:
        print(city)
        scores.append(np.nan)

odense
trondheim
västerås
aalborg
washington, dc
stavanger
los angeles
new york
düsseldorf
helsingborg
espoo
strasbourg
joensuu
frankfurt am main
san francisco
bochum
hämeenlinna
jyväskylä
vantaa
daejeon
lahti
abu dhabi
verona
catania
bayreuth
bari
perugia
tel aviv
hong kong
ancona
kuala lampur
genova
st petersburg


In [40]:
dic_quality = {}

for i in range (len(cities)):
    dic_quality.update({cities[i]:scores[i]})

In [41]:
smart["Life_Quality"]= smart["City"].map(dic_quality)

# 4. Download clean data into a new csv. file

In [25]:
smart.to_csv("../data/continent_smart.csv", index=False)